In [80]:
## CLEAR VARIABLES
# %reset

# Explanatory Data Analysis

## Objectives:

- Analysis of NA values
- Correlation Analysis
- Analysis of Categorical/Numerical values

## Load Data

In [131]:
import pandas as pd
import numpy as np
import sys
import os
import matplotlib.pyplot as plt
import seaborn as sns
from tabulate import tabulate

# Import data
train_x_raw = pd.read_csv("../01-Data/X_train.csv", low_memory = True, index_col=0)
train_y_raw = pd.read_csv("../01-Data/y_train.csv", low_memory = True, index_col=0)
test_x_raw = pd.read_csv("../01-Data/X_test.csv", low_memory=True, index_col=0)

df_train = pd.DataFrame(train_x_raw)
df_test = pd.DataFrame(test_x_raw)
df_y = pd.DataFrame(train_y_raw)

# display(df)
# df.describe().T

## Check Missing Data
- v228b_r: respondents country of birth (ISO 3166-1 numeric code) 
- v231b_r: fathers country of birth (ISO 3166-1 numeric code) 
- v233b_r: mothers country of birth (ISO 3166-1 numeric code) 
- v251b_r: spouse/partners country of birth (ISO 3166-1 numeric code) 

Note that they are translate from (ISO 3166-1/3 Alpha code) to (ISO 3166-1 numeric code). Thus, we need to also examine Alpha code variables as well.

- v228b: respondents country of birth (ISO 3166-1/3 Alpha code)
- v231b: fathers country of birth (ISO 3166-1/3 Alpha code)
- v233b: mothers country of birth (ISO 3166-1/3 Alpha code)
- v251b: spouse/partners country of birth (ISO 3166-1/3 Alpha code)

=> Hot-one encoding for ctry strs

## Important notes:
### Analysis (V111_4 ~ V228b_r)
- *v112 / f112_SE* (Same question: are you willing to fight for country): DROP ==> only -4 in train dataset
- Variables ... ending with _11c ex: *v133 / v133_11c* (Same question): -4 ==> Sum of all the other variables in *v133_11c*


In [82]:
Answer_table = [[-10, "multiple answers Mail"], [-2, "no answer"], [-1, "dont know"], [1, "a great deal"], [2, "quite a lot"],
                [3, "not very much"], [4, "none at all"]]
print(tabulate(Answer_table, headers=["Numeric Value", "Survey Answer"], tablefmt='github'))

|   Numeric Value | Survey Answer         |
|-----------------|-----------------------|
|             -10 | multiple answers Mail |
|              -2 | no answer             |
|              -1 | dont know             |
|               1 | a great deal          |
|               2 | quite a lot           |
|               3 | not very much         |
|               4 | none at all           |


In [135]:
############################################################# FUNCTIONS ###############################################################

### Function to find the targeted colname
def find_colname(data, target, fcn='startwith'):
    temp = []
    for varname in data.columns:
        if fcn == 'startwith':
          if varname.startwith(target):
            temp.append(varname)
        elif fcn == 'endwith':
           if varname.endwith(target):
            temp.append(varname)
    return(temp)

merge_colname = find_colname(train_x_raw, '_11c')
print(find_colname(train_x_raw, 'c'))
print(find_colname(train_x_raw, '_r'))

from collections import Counter

def merge_columns(dat, colname):
    for name in colname:
        name_org = name.replace("_11c", "")
        dat.loc[dat[name_org] == -4, name_org] = dat.loc[dat[name_org] == -4, name]


def print_diff(varname):
  print(set(df_train[varname].unique()).difference(set(df_test[varname].unique())))

def cumulatively_categorise(column,threshold=0.80,return_categories_list=True):
  #Find the threshold value using the percentage and number of instances in the column
  threshold_value=int(threshold*len(column))
  #Initialise an empty list for our new minimised categories 
  categories_list=[]
  #Initialise a variable to calculate the sum of frequencies
  s=0
  #Create a counter dictionary of the form unique_value: frequency
  counts=Counter(column)

  #Loop through the category name and its corresponding frequency after sorting the categories by descending order of frequency
  for i,j in counts.most_common():
    #Add the frequency to the global sum
    s+=dict(counts)[i]
    #Append the category name to the list
    categories_list.append(i)
    #Check if the global sum has reached the threshold value, if so break the loop
    if s>=threshold_value:
      break
  #Append the category Other to the list
  categories_list.append(-100)

  #Replace all instances not in our new categories by Other  
  new_column=column.apply(lambda x: x if x in categories_list else -100)

  #Return transformed column and unique values if return_categories=True
  if(return_categories_list):
    return new_column,categories_list
  #Return only the transformed column if return_categories=False
  else:
    return new_column
  
def simpleAggregation_helper(var, threshold):
  train=df_train[var]
  test=df_test[var]
  cat = [train, test]
  df = pd.concat(cat)
  transformed_column=cumulatively_categorise(df, threshold, return_categories_list=False)
  tc_train=transformed_column[0:len(train)]
  tc_test=transformed_column[len(train):len(train)+len(test)]
  df_train[var]=tc_train
  df_test[var]=tc_test

  
def simpleAggregation(variable, threshold=0.8):
    if isinstance(variable, str):
      simpleAggregation_helper(variable, threshold)
    elif isinstance(variable, list):
      for varname in variable:
        simpleAggregation_helper(varname, threshold)

##############################################################################################################################

AttributeError: 'str' object has no attribute 'startwith'

In [136]:
### Find variables containing _cs and do SimpleAggregation
print(find_colname(df_train, '_cs'))
simpleAggregation(find_colname(df_train, '_cs'))

AttributeError: 'str' object has no attribute 'startwith'

## Year/Month Duration Calculation

In [132]:
### Convert fw_start ==> Start month of fw
### Convert fw_end ==> Duration of fw
def timeEDA(data):
    fw_start = data['fw_start']
    fw_end = data['fw_end']
    fieldwork_start_month = []
    fw_duration = []
    for obs in range(0, len(fw_end)):
        fw_start_year = int(str(fw_start[obs])[0:4])
        fw_start_month = int(str(fw_start[obs])[4:6])
        fw_end_year = int(str(fw_end[obs])[0:4])
        fw_end_month = int(str(fw_end[obs])[4:6])
        duration_year = fw_end_year - fw_start_year
        duration_month = fw_end_month - fw_start_month
        duration = 12*duration_year + duration_month
        fieldwork_start_month.append(fw_start_month)
        fw_duration.append(duration)
    data['fw_start'] = fieldwork_start_month
    data['fw_end'] = fw_duration
    data.rename(columns={'fw_start':'fw_start_month', 'fw_end':'fw_duration'}, inplace=True)
timeEDA(df_train)

,year,fieldwork_start_month,fw_duration,country,c_abrv,v1,v2,v3,v4,v5,...,v278b,v278c_r,v279a,v279b,v279c_r,v279d_r,v280,v281a,v281a_r,v282
id,,,,,,,,,,,,,,,,,,,,,
0,2020,202011,202011,804,UA,2,1,2,2,3,...,45,14.45,15,40,15.40,55,1,uk,475,32
1,2018,201809,201811,688,RS,2,1,2,2,3,...,12,17.12,18,25,18.25,73,2,sr,397,3
2,2017,201706,201803,352,IS,2,2,1,1,2,...,58,15.58,13,20,13.20,-5,2,is,72,87
3,2021,202105,202110,428,LV,1,2,1,1,2,...,42,8.42,8,51,8.51,9,1,lv,256,5157
4,2018,201801,201805,40,AT,1,1,2,2,3,...,51,12.51,13,53,13.53,62,1,de,158,64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47995,2019,201812,201903,807,MK,1,1,1,2,2,...,10,14.10,14,48,14.48,38,1,mk,279,1
47996,2018,201808,201812,578,NO,1,1,1,2,2,...,20,15.20,16,45,16.45,85,1,no,200,1302
47997,2018,201802,201807,8,AL,2,1,1,2,4,...,40,13.40,14,20,14.20,40,1,sq,13,10
